In [1]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool


In [2]:

from langchain.utilities import GoogleSearchAPIWrapper

In [3]:
from tqdm.auto import tqdm
import datetime
from time import sleep


In [4]:
search = GoogleSearchAPIWrapper()
def fib(n):
    if n <= 1:
        return n 
    else:
        return (fib(n-1) + fib(n-2))
    
def sort_string(string):
    return ''.join(sorted(string))

def encypt(word):
    encrypted_word = ""
    for letter in word:
        encrypted_word += chr(ord(letter)+1)
    return encrypted_word

def decrypt(word):
    decrpted_word = ""
    for letter in word:
        decrypted_word += chr(ord(letter)-1)
    return(decrypted_word)

In [5]:
from uuid import uuid4

tools = [
    {
    "id" : str(uuid4()),
    "name" : "Google Search",
    "description":"Search Google for recent results.",
    "func":"search.run"
    },
    {
    "id" : str(uuid4()),
    "name" : "Fibonacci",
    "func": "lambda n: str(fib(int(n)))",
    "description" : "use when you want to calculate the nth fibonacci number",
    },
    {
    "id"  : str(uuid4()),
    "name"  : "Sort String",
    "func" : "lambda stuing: sort_string(string)",
    "description"  : "use when you want to sort a string alphabetically",
    },
    {
    "id" : str(uuid4()),
    "name" : "Encrypt",
    "func" : "lambda word :encrypt(word)",
    "description" : "use when you want to encrypt a word",
    },
    {
    "id":  str(uuid4()),
    "name":  "Decrypt",
    "func" : "lambda word :decrypt(word)",
    "description":  "use when you eant to decrypt a word",
    }


]

In [6]:
import openai
embed_model ="text-embedding-ada-002"


In [7]:
batch_size = 10
for i in tqdm(range(0, len(tools),batch_size)):
    i_end = min(len(tools),batch_size)
    meta_batch = tools[i:i_end]

    ids_batch = [x["id"] for x in meta_batch ]

    texts = [x['description'] for x in meta_batch]

    try:
        res = openai.Embedding.create(input=texts,engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts,engine=embed_model)
                done = True
            except:
                pass
    embeds=[record['embedding'] for record in res ['data']]

    meta_batch = [{
        'name': x['name'],
        'func': x['func'],
        'description': x['description']
    }for x in meta_batch]




  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
to_upsert = list(zip(ids_batch, embeds, meta_batch))

In [9]:
#to_upsert[1]

In [10]:
import pinecone
import os
index_name = 'try-langchain-tools'
index = pinecone.Index(index_name)

In [11]:
import pinecone
import os

index_name = 'try-langchain-tools'

# initialize connection to pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # app.pinecone.io (console)
    environment="us-central1-gcp"  # next to API key in console
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

In [12]:
index = pinecone.Index(index_name)

In [13]:
index.upsert(vectors=to_upsert)

{'upserted_count': 5}

In [148]:
query = "我需要python 的資訊 ?"

In [149]:
res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

In [150]:
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=1, include_metadata=True)

In [151]:
contexts = [item['metadata']['description'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [152]:
augmented_query

'Search Google for recent results.\n\n-----\n\n我需要python 的資訊 ?'

In [153]:
primer = f""" 
Please follow the instructions carefully and proceed step by step without doing anything else.
You are a robot that evaluates tools.
You will have a description of the tools and user's questions.
Your job is to evaluate whether the question can be solved based on the description.
And return a boolean value in JSON format.

Output Json Example:
{{
"result": True,

}}



"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    temperature = 0.3,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

In [154]:
from IPython.display import Markdown

ans = res['choices'][0]['message']['content']

In [155]:
ans = eval(ans)

In [156]:
ans

{'result': False}

In [157]:
if ans['result'] == True :
    print('有工具')
else :
    print("請說清楚問題")

請說清楚問題


'{\n"result": True\n}\n\nThe tool that can solve this problem is a function that calculates the nth Fibonacci number. This function takes an integer n as input and returns the nth Fibonacci number.'

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

{'matches': [{'id': '363cc02a-76f6-4d02-a51f-787bc57489f8',
              'metadata': {'description': 'use when you want to calculate the '
                                          'nth fibonacci number',
                           'func': 'lambda n: str(fib(int(n)))',
                           'name': 'Fibonacci'},
              'score': 0.683410764,
              'values': []},
             {'id': 'c088d51a-2e3c-470c-b8eb-ba499a2cfc5b',
              'metadata': {'description': 'use when you want to calculate the '
                                          'nth fibonacci number',
                           'func': 'lambda n: str(fib(int(n)))',
                           'name': 'Fibonacci'},
              'score': 0.683403909,
              'values': []}],
 'namespace': ''}
